### Leer el archivo JSON usando el "DataFrameReader" de Spark

### Ingestion de la carpeta "movie_language"

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
productions_countries_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True),
])

In [0]:
productions_countries_df = spark.read.schema(productions_countries_schema).option("multiLine", True).json(
    f"{bronze_folder_path}/{v_file_date}/production_country"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
productions_countries_final_df = productions_countries_df.withColumnsRenamed({
    'movieId': 'movie_id',
    'countryId': 'country_id',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.productions_countries")

overwrite_partition(productions_countries_final_df, "movie_silver", "productions_countries", "file_date")

In [0]:
#productions_countries_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/productions_countries")

#productions_countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.productions_countries")

productions_countries_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_countries")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies_casts

In [0]:
%sql
select file_date, count(1) from movie_silver.productions_countries group by file_date;